In [215]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
import requests
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import numpy as np
import optuna
import xgboost as xgb
from dateutil.easter import easter
from datetime import timedelta
import plotnine as p9

# Load

In [216]:
sales_daily = (
    pd.read_csv("./data/external/train.csv")
    .assign(

        date = lambda df_: pd.to_datetime(df_['date']),

        country_store = lambda df_: df_['country'].str.cat(df_['store'], sep='|'),
        country_product = lambda df_: df_['country'].str.cat(df_['product'], sep='|'),
        store_product = lambda df_: df_['store'].str.cat(df_['product'], sep='|'),
        country_store_product = lambda df_: df_['country'].str.cat([df_['store'], df_['product']], sep='|')

    )
    .assign(series_id = lambda df_: df_['country_store_product'])
)

In [ ]:
sales_daily.head()

In [ ]:
def extract_gdp_per_capita(country_code, year):
    """
    Adapted from https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554349.
    TODO: what about GDP level, which accounts also for population?

    Source reference: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation.
    """

    url='https://api.worldbank.org/v2/country/{0}/indicator/NY.GDP.PCAP.CD?date={1}&format=json'
    response = requests.get(url.format(country_code, year)).json()

    return response[1][0]['value']

# per CountryCode-year: request GDP per capita.
# concatenate dataframe of CountryCode | Country | Year | GDP, for integration to Kaggle source

countries_code_map = {
    'Canada': 'CAN', 
    'Finland': 'FIN',
    'Italy': 'ITA',
    'Kenya': 'KEN',
    'Norway': 'NOR',
    'Singapore': 'SGP'
    }

countries_gdp_yearly = []
for country_title, country_code in countries_code_map.items():
    
    values_yearly = [
        {'year': i, 'gdp_per_capita': extract_gdp_per_capita(country_code, i)}
        for i in range(2010, 2019+1)
        ]
    values_yearly = [pd.DataFrame(x, index=[0]) for x in values_yearly]
    values_yearly = pd.concat(values_yearly, axis=0)
    
    values_yearly = (
        values_yearly
        .assign(
            country = country_title,
            country_code = country_code
            )
        .sort_values('year')
        .assign(
            gdp_per_capita_lag1 = lambda df_: df_['gdp_per_capita'].shift(1),
            gdp_per_capita_lag2 = lambda df_: df_['gdp_per_capita'].shift(2),
            gdp_per_capita_lag3 = lambda df_: df_['gdp_per_capita'].shift(3)
            )
        .assign(
            # first observation won't have a preceding lag
            is_recession = lambda df_: (df_['gdp_per_capita'] < df_['gdp_per_capita_lag1']).astype(int).fillna(0),
            gdp_per_capita_growth1 = lambda df_: np.log(df_['gdp_per_capita']) - np.log(df_['gdp_per_capita_lag1']),
            gdp_per_capita_growth2 = lambda df_: np.log(df_['gdp_per_capita']) - np.log(df_['gdp_per_capita_lag2']),
            gdp_per_capita_growth3 = lambda df_: np.log(df_['gdp_per_capita']) - np.log(df_['gdp_per_capita_lag3'])
            )
        .assign(
            # asymmetric effects of growth & contraction
            gdp_per_capita_growth1_is_positive = lambda df_: np.where(df_['gdp_per_capita_growth1'] > 0, df_['gdp_per_capita_growth1'], 0),
            gdp_per_capita_growth1_is_negative = lambda df_: np.where(df_['gdp_per_capita_growth1'] < 0, df_['gdp_per_capita_growth1'], 0),

            gdp_per_capita_growth2_is_positive = lambda df_: np.where(df_['gdp_per_capita_growth2'] > 0, df_['gdp_per_capita_growth2'], 0),
            gdp_per_capita_growth2_is_negative = lambda df_: np.where(df_['gdp_per_capita_growth2'] < 0, df_['gdp_per_capita_growth2'], 0),

            gdp_per_capita_growth3_is_positive = lambda df_: np.where(df_['gdp_per_capita_growth3'] > 0, df_['gdp_per_capita_growth3'], 0),
            gdp_per_capita_growth3_is_negative = lambda df_: np.where(df_['gdp_per_capita_growth3'] < 0, df_['gdp_per_capita_growth3'], 0),
        )
        )
    
    countries_gdp_yearly.append(values_yearly)

    print(f"{country_title} ({country_code}) GDP Per Capita extraction complete.")

countries_gdp_yearly = pd.concat(countries_gdp_yearly, axis=0)

countries_gdp_yearly = (
    countries_gdp_yearly
    .assign(gdp_per_capita_log = lambda df_: np.log(df_['gdp_per_capita']))
    )
countries_gdp_yearly['gdp_per_capita_log^2'] = countries_gdp_yearly['gdp_per_capita_log']**2

In [ ]:
countries_gdp_yearly

In [ ]:
(
    countries_gdp_yearly
    [['year', 'country', 'is_recession', 'gdp_per_capita']]
    .set_index(['country', 'year'])
    .unstack(-1)
)

In [ ]:
(
    countries_gdp_yearly
    .set_index('year')
    .groupby('country')
    ['gdp_per_capita']
    .plot(legend=True, marker='o')
)
;

In [ ]:
(
    countries_gdp_yearly
    .query("country == 'Kenya'")
    .set_index('year')
    .groupby('country')
    ['gdp_per_capita']
    .plot(legend=True)
)
;

In [ ]:
def extract_population(country_code, year):

    url='https://api.worldbank.org/v2/country/{0}/indicator/SP.POP.TOTL?date={1}&format=json'
    response = requests.get(url.format(country_code, year)).json()

    return response[1][0]['value']

# concatenate dataframe of CountryCode | Country | Year | KPI, for integration to Kaggle source

countries_population_yearly = []
for country_title, country_code in countries_code_map.items():
    
    values_yearly = [
        {'year': i, 'population': extract_population(country_code, i)}
        for i in range(2010, 2019+1)
        ]
    values_yearly = [pd.DataFrame(x, index=[0]) for x in values_yearly]
    values_yearly = pd.concat(values_yearly, axis=0)
    
    values_yearly = (
        values_yearly
        .assign(
            country = country_title,
            country_code = country_code
            )
        )
    
    countries_population_yearly.append(values_yearly)

    print(f"{country_title} ({country_code}) population extraction complete.")

countries_population_yearly = pd.concat(countries_population_yearly, axis=0)
assert countries_population_yearly.notnull().all().all()

countries_population_yearly = (
    countries_population_yearly
    .assign(population_log = lambda df_: np.log(df_['population']))
    )
countries_population_yearly['population_log^2'] = countries_population_yearly['population_log']**2

In [ ]:
(
    countries_population_yearly
    .set_index('year')
    .groupby('country')
    ['population_log']
    .plot(legend=True, marker='o')
)
;

In [11]:
# TODO: is there a better practice for holidays feature representation?
    # in Hyndman's Electricity Load Forecasting Kaggle (https://robjhyndman.com/papers/kaggle-competition.pdf),
    # "holiday effect modelled with a factor variable, taking value zero on a non-work day,
    # some non-zero value day before a non-work day, and a different value the day after a non-work day."
    # meaning -- holiday days pooled, before & after estimated separately?

def onehot_encode_dates_delta(dates_base_feature: pd.DataFrame, deltas_day: list):
    """
    Given dates where `is_easter` (dates_base_feature): 
    - create the Easter-1 days, title `is_easter_sub1day`.
    - create the Easter+1 days, title `is_easter_add1day`.
    ... So on, for deltas_day beyond [-1, 1].

    Motivated by analysis of model errors.
    """

    base_feature_stem = list(dates_base_feature.columns)
    base_feature_stem.remove('date')
    base_feature_stem = base_feature_stem[0]

    frames_date_deltas = [dates_base_feature]
    for delta in deltas_day: 

        if delta < 0:
            sgn = 'sub'
        else:
            sgn = 'add'

        df_delta = (
            dates_base_feature
            .copy()
            .assign(date = lambda df_: df_['date'] + timedelta(days=delta))
            .rename(columns={base_feature_stem: f"{base_feature_stem}_{sgn}{abs(delta)}"})
            )
        
        frames_date_deltas.append(df_delta)

    dates_features = (
        pd.concat(frames_date_deltas, axis=0)
        .fillna(0)
        .assign(date = lambda df_: pd.to_datetime(df_['date']))
        )

    assert dates_features['date'].is_unique

    return dates_features


days_easter0 = [easter(x) for x in range(2010, 2019+1)]
days_easter = pd.DataFrame({'date': days_easter0}).assign(is_easter = 1)
days_special_relative_easter = onehot_encode_dates_delta(days_easter, [i for i in range(2, 7+1)])
FEATURES_EASTER = list(days_special_relative_easter.columns)
FEATURES_EASTER.remove('date')

days_new_years_eve = [pd.to_datetime(f"{yr}-12-31") for yr in range(2010, 2019+1)]
days_new_years_eve = pd.DataFrame({'date': days_new_years_eve}).assign(is_new_years_eve = 1)
days_special_relative_new_years_eve = onehot_encode_dates_delta(
    days_new_years_eve, 
    [-3, -2, -1, 1, 2, 3]
    )
FEATURES_NEW_YEARS_EVE = list(days_special_relative_new_years_eve.columns)
FEATURES_NEW_YEARS_EVE.remove('date')

In [12]:
def transform_calendar_features(df):

    df = (
        df
        .assign(
            year = lambda df_: df_['date'].dt.year,
            month = lambda df_: df_['date'].dt.month,
            week_of_year = lambda df_: df_['date'].dt.isocalendar().week,
            day_of_week = lambda df_: df_['date'].dt.day_name(),
            # President's Day is the 'third Monday in February'
            day_of_month = lambda df_: df_['date'].dt.day,
            day_of_year = lambda df_: df_['date'].dt.dayofyear,
            # week of month would be ambiguous because, one week may span 2 months,
            days_since_start = lambda df_: (df['date'] - pd.to_datetime("2010-01-01")).dt.days
            )
        .assign(
            # TODO: are periodic feature transforms too rigid?

            # as day_of_year rises, don't expect monotonic relationship with outcome.
            # rather, expect periodic (sinusoidal) relationship.
            # as sin(x) rises, so too does outcome ...
            # ensure one cycle over one year.
            # at baseline, one sinusoidal cycle occurs per 2π
            day_of_year_sin = lambda df_: np.sin(df_['day_of_year'] * 2 * np.pi / 365),
            day_of_year_cos = lambda df_: np.cos(df_['day_of_year'] * 2 * np.pi / 365),

            day_of_month_sin = lambda df_: np.sin(df_['day_of_month'] * 2 * np.pi / 30),
            day_of_month_cos = lambda df_: np.cos(df_['day_of_month'] * 2 * np.pi / 30),

            # exploratory visuals suggest multi-year cycles.
            # moreover, for out-of-sample predictions to follow a periodic dynamic,
            # need a strong parameterized form like this.
            days_since_start_2year_sin = lambda df_: np.sin(df_['days_since_start'] * 2 * np.pi / (365*2)),
            days_since_start_3year_sin = lambda df_: np.sin(df_['days_since_start'] * 2 * np.pi / (365*3)),
            days_since_start_4year_sin = lambda df_: np.sin(df_['days_since_start'] * 2 * np.pi / (365*4)),
            days_since_start_5year_sin = lambda df_: np.sin(df_['days_since_start'] * 2 * np.pi / (365*5)),

            days_since_start_2year_cos = lambda df_: np.cos(df_['days_since_start'] * 2 * np.pi / (365*2)),
            days_since_start_3year_cos = lambda df_: np.cos(df_['days_since_start'] * 2 * np.pi / (365*3)),
            days_since_start_4year_cos = lambda df_: np.cos(df_['days_since_start'] * 2 * np.pi / (365*4)),
            days_since_start_5year_cos = lambda df_: np.cos(df_['days_since_start'] * 2 * np.pi / (365*5)),
            )

        )
    
    df = pd.merge(df, days_special_relative_easter, how='left')
    assert df['is_easter'].notnull().any()
    df[FEATURES_EASTER] = df[FEATURES_EASTER].fillna(0)

    df = pd.merge(df, days_special_relative_new_years_eve, how='left')
    assert df['is_new_years_eve'].notnull().any()
    df[FEATURES_NEW_YEARS_EVE] = df[FEATURES_NEW_YEARS_EVE].fillna(0)
    
    return df

def integrate_external_features(df):

    df = pd.merge(df, countries_gdp_yearly, how='left')
    assert df['gdp_per_capita'].notnull().all().all()

    df = pd.merge(df, countries_population_yearly, how='left')
    assert df['population'].notnull().all().all()

    return df

def transform_lagged_predictors(df):

    # to ensure proper within-series outcome lags
    # TODO: with lagged features coming into play, how to enforce proper order via indexes?
    df = (
        df
        .sort_values(['series_id', 'date'])
        .assign(
            num_sold_lag1 = lambda df_: df_.groupby('series_id')['num_sold'].shift(1),
            num_sold_lag7 = lambda df_: df_.groupby('series_id')['num_sold'].shift(7)
            )
        )
    
    return df

def transform_logs(df):

    df = (
        df
        .assign(
            num_sold_log = lambda df_: np.log(df_['num_sold']),
            num_sold_lag1_log = lambda df_: np.log(df_['num_sold_lag1']),
        )
    )

    return df

sales_daily = transform_calendar_features(sales_daily)
sales_daily = integrate_external_features(sales_daily)
sales_daily = transform_lagged_predictors(sales_daily)
sales_daily = transform_logs(sales_daily)

# Data Understanding

## Data Description Report: "Surface Properties"

In [ ]:
sales_daily['id'].is_unique

### Volumetric Analyses

In [ ]:
sales_daily.shape

In [ ]:
sales_daily['series_id'].nunique()

In [ ]:
sales_daily['date'].nunique()

In [ ]:
sales_daily['series_id'].value_counts().value_counts()

In [ ]:
sales_daily['product'].value_counts(dropna=False)

In [ ]:
sales_daily['country'].value_counts(dropna=False)

In [ ]:
sales_daily['store'].value_counts(dropna=False)

### Fields' Types and Values

In [ ]:
sales_daily['date'].describe()

In [ ]:
sales_daily['num_sold'].describe()

## Data Quality Report

In [ ]:
sales_daily.isnull().mean()

In [ ]:
# are null sales events concentrated on a particular date?
# doesn't appear so
sales_daily.query("num_sold.isnull()")['date'].value_counts()

In [ ]:
sales_daily.query("num_sold.isnull()")['series_id'].value_counts()

## Data Exploration Report

In [ ]:
sales_daily.groupby('series_id')['num_sold'].sum().sort_values(ascending=False)

In [ ]:
# a naive model: series' historical average sales, daily

is_training = (
    (sales_daily['date'] >= pd.to_datetime('2010-01-01'))
    & (sales_daily['date'] < pd.to_datetime("2014-01-01"))
)

is_validation = sales_daily['date'] >= pd.to_datetime("2014-01-01")

# is_training.sum(), is_validation.sum()

# with original train data: using strictly train segment, groupby average
predictions_naive = (
    sales_daily
    .loc[is_training]
    .groupby('series_id')
    [['num_sold']]
    .agg('mean')
    .reset_index(drop=False)
)

predictions_naive_evaluate = pd.merge(
    sales_daily.loc[is_validation],
    predictions_naive.rename(columns={'num_sold': 'yhat'}),
    how='left'
    )

# expect a couple series with all null
predictions_naive.isnull().sum()
# recommended in this discussion: https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554553 
predictions_naive_evaluate = predictions_naive_evaluate.dropna()

mean_absolute_percentage_error(
    predictions_naive_evaluate['num_sold'],
    predictions_naive_evaluate['yhat']
)

In [ ]:
predictions_naive_evaluate.shape

In [ ]:
# a naive model: series' historical average sales, daily

is_training = (
    (sales_daily['date'] >= pd.to_datetime('2013-01-01'))
    & (sales_daily['date'] < pd.to_datetime("2014-01-01"))
)

is_validation = sales_daily['date'] >= pd.to_datetime("2014-01-01")

# is_training.sum(), is_validation.sum()

# with original train data: using strictly train segment, groupby average
predictions_naive = (
    sales_daily
    .loc[is_training]
    .groupby('series_id')
    [['num_sold']]
    .agg('mean')
    .reset_index(drop=False)
)

predictions_naive_evaluate = pd.merge(
    sales_daily.loc[is_validation],
    predictions_naive.rename(columns={'num_sold': 'yhat'}),
    how='left'
    )

# expect a couple series with all null
predictions_naive.isnull().sum()
# recommended in this discussion: https://www.kaggle.com/competitions/playground-series-s5e1/discussion/554553 
predictions_naive_evaluate = predictions_naive_evaluate.dropna()

mean_absolute_percentage_error(
    predictions_naive_evaluate['num_sold'],
    predictions_naive_evaluate['yhat']
)

In [ ]:
sales_sample_daily = sales_daily.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

(
    sales_sample_daily
    .loc[is_training]
    [['date', 'num_sold']]
    .set_index('date')
    .plot
    .line()
)
;

In [31]:
# descend level of abstraction -- 

    # across-year patterns (multi-year business cycles)
    # within-year,
        # month-of-year seasonality
        # week-of-year seasonality
        # day-of-month seasonality
        # day-of-week seasonality

# outcome vs predictors
    # lagged outcome
    # gdp

In [ ]:
(
    p9.ggplot(sales_daily.groupby(['year'])[['num_sold']].agg('sum').reset_index(drop=False)) + 
    p9.theme_bw() + 
    p9.geom_col(p9.aes('year', 'num_sold'), alpha=0.25)
)

In [ ]:
(
    p9.ggplot(sales_daily.groupby(['country', 'year'])[['num_sold']].agg('sum').reset_index(drop=False)) + 
    p9.theme_bw() + 
    p9.geom_line(p9.aes('year', 'num_sold', group='country', color='country')) + 
    p9.theme(figure_size=(8,4))
)

In [ ]:
(
    p9.ggplot(sales_daily.groupby(['date'])[['num_sold']].agg('sum').reset_index(drop=False)) + 
    p9.theme_bw() + 
    p9.geom_point(p9.aes('date', 'num_sold'), alpha=0.25)
)

In [ ]:
(
    p9.ggplot(sales_daily) + 
    p9.theme_bw() + 
    p9.geom_boxplot(p9.aes('month', 'num_sold', group='month'), alpha=0.1)
)

In [ ]:
(
    p9.ggplot(sales_daily.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")) + 
    p9.theme_bw() + 
    p9.geom_boxplot(p9.aes('month', 'num_sold', group='month'), alpha=0.1)
)

In [ ]:
(
    p9.ggplot((
        sales_daily
        .assign(day_of_week = lambda df_: pd.Categorical(
            df_['day_of_week'], 
            ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
            ))
    )) + 
    p9.theme_bw() + 
    p9.geom_boxplot(p9.aes('day_of_week', 'num_sold', group='day_of_week'), alpha=0.1)
)

In [ ]:
(
    p9.ggplot(sales_daily.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")) + 
    p9.theme_bw() + 
    p9.geom_point(p9.aes('num_sold_lag1', 'num_sold'), alpha=0.1)
)

In [ ]:
(
    p9.ggplot(sales_daily.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")) + 
    p9.theme_bw() + 
    p9.geom_point(p9.aes('num_sold_lag7', 'num_sold'), alpha=0.1)
)

In [ ]:
(
    p9.ggplot((
        sales_daily
        .assign(num_sold_log_mean = lambda df_: df_.groupby('series_id')['num_sold_log'].transform('mean'))
        .assign(num_sold_log_demean = lambda df_: df_['num_sold_log'] - df_['num_sold_log_mean'])
        )) + 
    p9.theme_bw() + 
    p9.geom_point(p9.aes('gdp_per_capita_log', 'num_sold_log_demean'), alpha=0.5) + 
    p9.geom_smooth(p9.aes('gdp_per_capita_log', 'num_sold_log_demean', group='country', color='country'), method='lowess')
)

# Data Preparation

Features set varies by model -- _global_ and _local_. 

"Universe" implies, all features that could be used. A subset enters into each "building block" of the overall modeling system.

In [41]:
FEATURES_UNIVERSE_TO_ONEHOT = [
    'country', 
    'store',
    'product',
    'country_store',
    'country_product',
    'store_product',
    # omit country-store-product, that's perfect collinearity with above.
    # year attempted, but then omitted. Because 
    # exogenous factors should help explain year-to-year shifts,
    # so that out-of-sample years' forecasts aren't flat
    'month', 
    'week_of_year', 
    'day_of_week'
    ]

# local model fits by country-store-product segment;
# therefore, those onehots don't vary
FEATURES_LOCAL_MODEL_TO_ONEHOT = [
    x for x in FEATURES_UNIVERSE_TO_ONEHOT 
    if not any(stem in x for stem in ['country', 'store', 'product'])
    ]

FEATURES_UNIVERSE_NUMERIC_CONTINUOUS = [
    'gdp_per_capita_log',
    'gdp_per_capita_log^2',
    'population_log',
    'population_log^2',

    # continuous year allows evolving patterns over time
    'year',
    # TODO: better treated as onehot?
    'day_of_month', 
    'day_of_month_sin',
    'day_of_month_cos',

    'day_of_year',
    'day_of_year_sin',
    'day_of_year_cos',

    'days_since_start',
    'days_since_start_2year_sin',
    'days_since_start_3year_sin',
    'days_since_start_4year_sin',
    'days_since_start_5year_sin',
    'days_since_start_2year_cos',
    'days_since_start_3year_cos',
    'days_since_start_4year_cos',
    'days_since_start_5year_cos'
    ]
# from experimentation, tree-based models' out-of-sample forecasts flat
# when incorporating days_since_start based feature. (or onehot-encoded year)

FEATURES_UNIVERSE_ALREADY_ONEHOT = FEATURES_EASTER + FEATURES_NEW_YEARS_EVE

FEATURES_AUTOREGRESSIVE = ['num_sold_lag1_log']

FEATURES_GLOBAL_MODEL = (
    FEATURES_UNIVERSE_TO_ONEHOT + 
    FEATURES_UNIVERSE_NUMERIC_CONTINUOUS + 
    FEATURES_UNIVERSE_ALREADY_ONEHOT
    )

FEATURES_LOCAL_MODEL = (
    FEATURES_LOCAL_MODEL_TO_ONEHOT + 
    FEATURES_UNIVERSE_NUMERIC_CONTINUOUS + 
    FEATURES_UNIVERSE_ALREADY_ONEHOT
    )

ATTRIBUTES = ['series_id', 'date', 'id']

In [42]:
sales_daily_complete = sales_daily.dropna(subset=['num_sold', 'num_sold_lag1_log'])

# shorter alias
XY = sales_daily_complete

# from previous retail forecasting competitions' leaders,
# plus theoretically expected heterogeneity between series: 
# one model per segment

segments_XY = {grp: df for grp, df in XY.groupby('series_id')}

In [43]:
# have gone back-and-forth about the validation set decision.
# validation set 2014-16 ensures, more consistently accurate model (trend-cycle component, especially).
# validation 2016 ensures, higher accuracy on data very recent to 2017-19 test.
    # *if relationships are time-varying*, and sample size insufficient to explicitly estimate time-varying parameters -- 
    # then prefer to upweight data more recent to test period. 
KFOLDS_ALTERNATIVES = {

    'validate_2014_2016': ( 
        ( (XY['date'] >= pd.to_datetime('2010-01-01')) & (XY['date'] < pd.to_datetime("2014-01-01")) ),
        XY['date'] >= pd.to_datetime("2014-01-01")
        ),

    'validate_2015_2016': ( 
        ( (XY['date'] >= pd.to_datetime('2010-01-01')) & (XY['date'] < pd.to_datetime("2015-01-01")) ),
        XY['date'] >= pd.to_datetime("2015-01-01")
        ),

    'validate_2016': ( 
        ( (XY['date'] >= pd.to_datetime('2010-01-01')) & (XY['date'] < pd.to_datetime("2016-01-01")) ),
        XY['date'] >= pd.to_datetime("2016-01-01")
        )

    }

In [62]:
# to execute finer-grained feature selection -- for example, which categorical levels to onehot-encode --
# start with pre-transformed, wide-as-possible features set

# must not leak train/test, and also expect model-building by segment.

# alternatives:
    # (1) pre-build train/test splits for every segment
        # drawbacks: data management complexity. many more objects to keep track of.
    # (2) pre-build one "universe features" dataset, to collect all those colnames.
    # then, on the fly, apply transformer and subset challenger cols.
        # drawbacks: repeating a calculation many times, when it could calculate upfront.

is_training, is_validation = KFOLDS_ALTERNATIVES['validate_2016']

segments_XY_transform_features_universe = {}
for grp, XY_grp in segments_XY.items():

    feature_transform_pipeline_universe = ColumnTransformer(
        [
            ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
            ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
            ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
        ],
        verbose_feature_names_out=False,
        remainder='drop'
        ).set_output(transform='pandas')

    X_train_transform = feature_transform_pipeline_universe.fit_transform(XY_grp.loc[is_training])
    y_train = XY_grp.loc[is_training, ['num_sold', 'num_sold_log']]
    X_test_transform = feature_transform_pipeline_universe.transform(XY_grp.loc[is_validation])
    y_test = XY_grp.loc[is_validation, ['num_sold', 'num_sold_log']]

    XY_grp_train = pd.concat([X_train_transform, y_train], axis=1)
    XY_grp_test = pd.concat([X_test_transform, y_test], axis=1)
    XY_grp = pd.concat([XY_grp_train, XY_grp_test], axis=0)

    segments_XY_transform_features_universe[grp] = {
        'X_train': X_train_transform,
        'y_train': y_train,
        'X_test': X_test_transform,
        'y_test': y_test,
        'XY_grp': XY_grp
        }

# Modeling

In [45]:
from typing import Literal

class TrendRemainderModelPipeline:
    """
    Y = TrendComponent + RemainderComponent
    """
    def __init__(
        self, 
        trend_level_feature_transformer: ColumnTransformer,
        trend_model_ridge_alpha,
        remainder_feature_transformer: ColumnTransformer,
        remainder_model_kind: Literal["random_forest", "xgboost"],
        remainder_model_xgboost_rounds_count=1_000,
        remainder_model_xgboost_param=None
        ):

        self.trend_level_feature_transformer = trend_level_feature_transformer
        self.trend_level_model_ridge_alpha = trend_model_ridge_alpha

        self.remainder_feature_transformer = remainder_feature_transformer
        self.remainder_model_kind = remainder_model_kind
        
        self.remainder_model_xgboost_rounds_count = remainder_model_xgboost_rounds_count
        self.remainder_model_xgboost_param = remainder_model_xgboost_param

    def fit(self, X, y):

        self.fit_trend_level_model(X, y)
        yhat_trend_level = self.predict_trend_level_model(X)
        y_detrended = y - yhat_trend_level

        self.fit_remainder_model(X, y_detrended)

        return self

    def predict(self, X):

        yhat_trend_level = self.predict_trend_level_model(X)
        yhat_remainder = self.predict_remainder_model(X)
        preds = yhat_trend_level + yhat_remainder

        return preds

    def fit_trend_level_model(self, X, y):
        """
        'Weak learner', strictly intended to explain trend (level) shifts between years.
        """

        X = self.trend_level_feature_transformer.fit_transform(X)

        model_trend_level = Ridge(self.trend_level_model_ridge_alpha)
        model_trend_level.fit(X, y)

        self.trend_level_model = model_trend_level

    def _fit_xgboost_remainder_model(self, X, y):
        """
        Use xgboost-optimized data structures.
        Precondition: transformed X
        """

        dtrain = xgb.DMatrix(X, label=y)

        self.remainder_model = xgb.train(
            self.remainder_model_xgboost_param, 
            dtrain, 
            self.remainder_model_xgboost_rounds_count
            )

    def _fit_random_forest_remainder_model(self, X, y):
        """Precondition: transformed X"""

        self.remainder_model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
        self.remainder_model.fit(X, y)

    def fit_remainder_model(self, X, y):

        X = self.remainder_feature_transformer.fit_transform(X)

        if self.remainder_model_kind == 'xgboost':
            self._fit_xgboost_remainder_model(X, y)

        elif self.remainder_model_kind == 'random_forest':
            self._fit_random_forest_remainder_model(X, y)

    def predict_trend_level_model(self, X):
        X = self.trend_level_feature_transformer.transform(X)
        preds = self.trend_level_model.predict(X) 
        return preds
    
    def predict_remainder_model(self, X):

        X = self.remainder_feature_transformer.transform(X)
        if self.remainder_model_kind == 'xgboost':
            X = xgb.DMatrix(X)

        preds = self.remainder_model.predict(X)

        return preds

## Local Models

### Trend-Cycle Component: Strongly Parametric

In [ ]:
# for model to use gdp_per_capita_log (annual measurement),
# cannot include categorical year. otherwise, year partials out gdp

# experimented with gdp_per_capita_growth lags 1-3,
# but these *severely* overfit! 

# experimented with polynomial transforms of gdp and/or population,
# but those hardly improved result

# experimented with days_since_start_2year_sin, cos; but 
# result seems weakly improved versus additional model complexity

# TODO: some feature for expansionary vs contractionary periods?

FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS = [
    'gdp_per_capita_log',
    ]

def objective(trial):

    parameters = {'alpha': trial.suggest_float("alpha", 1e-1, 1_000, log=True)}

    kfolds_evaluation = []
    for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

        segments_models = {}
        segments_predictions = []

        for grp, XY_grp in segments_XY.items():

            feature_transform_pipeline_local_model = ColumnTransformer([
                # ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['year']),
                ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
                ],
                verbose_feature_names_out=False,
                remainder='drop'
                )
            feature_transform_pipeline_local_model.set_output(transform='pandas')

            pipeline_e2e = Pipeline([
                ('transform_features', feature_transform_pipeline_local_model), 
                ('model', Ridge(**parameters))
                ])
            pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])
            
            segments_models[grp] = pipeline_e2e

            predictions = (
                XY_grp
                .copy()
                .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))
                )
            
            segments_predictions.append(predictions)


        predictions = pd.concat(segments_predictions, axis=0)
        scores = {
            'validation': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                ),
            'train': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        
        kfolds_evaluation.append(scores)

    score_overall = np.mean([ 
        kfolds_evaluation[0]['validation'], 
        # kfolds_evaluation[1]['validation'] 
        ])
    
    return score_overall

study = optuna.create_study()

study.optimize(
    objective,
    n_trials=50,
    catch=(ValueError,),
    n_jobs=-1,
    timeout=12 * 60 * 60,
)

RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS = study.best_params

In [ ]:
study.best_trial, study.best_params, study.best_value

### Trend-Cycle Component: Periodicity Feature Selection 

Overfit, likely due to feature selection based on single validation set.

In [ ]:
# previously, overfit with boosting pipeline of:  
#   1. parametric trend level shifts
#   2. nonparametric periodic trend
#   3. nonparametric seasonality & remainder
# try (2) with a parametric model.

HAS_TREND_PERIODICITY_FEATURE_SELECTION = True
FEATURES_TREND_PERIODICITY_NUMERIC_CONTINUOUS = [
    # don't force equal magnitude across years.
    'days_since_start_2year_sin',
    'days_since_start_3year_sin',
    'days_since_start_4year_sin',
    'days_since_start_5year_sin',
    'days_since_start_2year_cos',
    'days_since_start_3year_cos',
    'days_since_start_4year_cos',
    'days_since_start_5year_cos'
    ]
features_champion = FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS
challengers_marginal_features_roots = [2, 3, 4, 5]
selections_model_alternatives = {-1: 0, 0: 0, 1: 0, 2:0, 3:0}

kfolds_evaluation = []
for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

    SEGMENTS_TREND_MODELS_TRAINSET = {}
    segments_predictions = []

    i = 0
    segments_total = len(segments_XY)
    for grp, XY_grp in segments_XY.items():

        feature_transform_pipeline_champion = ColumnTransformer(
            [('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)],
            verbose_feature_names_out=False,
            remainder='drop'
            ).set_output(transform='pandas')

        pipeline_e2e = Pipeline([
            ('transform_features', feature_transform_pipeline_champion), 
            ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
            ])
        pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])
        
        pipeline_champion_e2e = pipeline_e2e
        predictions_champion = XY_grp.copy().assign(yhat = lambda df_: np.exp(pipeline_champion_e2e.predict(df_)))

        score_champion = mean_absolute_percentage_error(
            predictions_champion.loc[is_validation, 'num_sold'],
            predictions_champion.loc[is_validation, 'yhat']
            )
        
        if HAS_TREND_PERIODICITY_FEATURE_SELECTION:

            models_challengers = []
            predictions_challengers = []
            scores_challengers = []
            for root in challengers_marginal_features_roots:

                features_marginal = [f"days_since_start_{root}year_sin", f"days_since_start_{root}year_cos"]
                features_challenger = features_champion + features_marginal

                feature_transform_pipeline_challenger = ColumnTransformer(
                    [('transformer_std', StandardScaler(), features_challenger)],
                    verbose_feature_names_out=False,
                    remainder='drop'
                    ).set_output(transform='pandas')
                pipeline_challenger_e2e = Pipeline([
                    ('transform_features', feature_transform_pipeline_challenger), 
                    ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
                    ])
                pipeline_challenger_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])
                models_challengers.append(pipeline_challenger_e2e)

                predictions_challenger = (
                    XY_grp
                    .copy()
                    .assign(yhat = lambda df_: np.exp(pipeline_challenger_e2e.predict(df_)))
                    )
                predictions_challengers.append(predictions_challenger)

                score_challenger = mean_absolute_percentage_error(
                    predictions_challenger.loc[is_validation, 'num_sold'],
                    predictions_challenger.loc[is_validation, 'yhat']
                    )
                scores_challengers.append(score_challenger)

            score_best_challenger = min(scores_challengers)
            is_best_challenger_better_vs_champion = score_best_challenger < score_champion
            if is_best_challenger_better_vs_champion:
                index_best = scores_challengers.index(score_best_challenger)
                model_final = models_challengers[index_best]            
                predictions_final = predictions_challengers[index_best]
                selections_model_alternatives[index_best] += 1
            else:
                model_final = pipeline_champion_e2e
                predictions_final = predictions_champion
                selections_model_alternatives[-1] += 1

        else:
            model_final = pipeline_champion_e2e
            predictions_final = predictions_champion

        SEGMENTS_TREND_MODELS_TRAINSET[grp] = model_final

        segments_predictions.append(predictions_final)
        
        i += 1
        print(f"{i}/{segments_total} models fit.")

    predictions = pd.concat(segments_predictions, axis=0)

    scores = {
        'validation': {
            'nobs': predictions.loc[is_validation].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                )
            },
        'train': {
            'nobs': predictions.loc[is_training].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        }
    
    kfolds_evaluation.append(scores)

kfolds_evaluation

In [ ]:
selections_model_alternatives

In [50]:
predictions = (
    predictions
    .assign(ape = lambda df_: abs(100 * (df_['num_sold'] / df_['yhat'] - 1)))
    .assign(pe = lambda df_: (100 * (df_['num_sold'] / df_['yhat'] - 1)))
    .assign(residual = lambda df_: df_['num_sold'] - df_['yhat'])
    .assign(residual_log = lambda df_: df_['num_sold_log'] - np.log(df_['yhat']))
    .assign(residual_abs = lambda df_: np.abs(df_['residual']))
)

In [ ]:
(
    predictions
    .groupby('country')
    ['ape']
    .agg(['mean', 'size'])
)

In [ ]:
(
    predictions
    .groupby('year')
    ['ape']
    .agg(['mean', 'size'])
)

In [ ]:
(
    predictions
    .groupby(['country', 'year'])
    ['ape']
    .agg(['mean', 'size'])
    .unstack(-1)
)

In [54]:
# looked previously at AVG(residual) by year,
# but that allows large errors to wash out -- not the right metric for this contest!

In [ ]:
predictions_sample = predictions.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line()

In [ ]:
predictions_sample = predictions.query("series_id == 'Kenya|Stickers for Less|Holographic Goose'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line()

In [ ]:
(
    predictions_sample
    .groupby('year')
    [['residual']]
    .agg([np.std, 'mean'])
)

### Seasonal & Remainder Component: Nonparametric, Explore Params

In [58]:
FEATURES_SEASONALITY_TO_ONEHOT = [
    'month', 
    'week_of_year',
    'day_of_week',
    ]

FEATURES_SEASONALITY_NUMERIC_CONTINUOUS = [
    'year', # allow seasonality to evolve over time
    'day_of_month', 
    'day_of_month_sin',
    'day_of_month_cos',
    'day_of_year_sin',
    'day_of_year_cos',
    'day_of_year'
    # previously tested days_since_start_{X}year_{sin_or_cos} -- forecasts worsened
    ]

In [59]:
# def objective(trial):

#     ROUNDS_COUNT = 1_000

#     param = {
#         "objective": "reg:squarederror",
#         "booster": "gbtree",
#         "max_depth": trial.suggest_int("max_depth", 1, 9, step=2),
#         "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         }

#     kfolds_evaluation = []
#     for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

#         segments_models = {}
#         segments_predictions = []

#         for grp, XY_grp in segments_XY.items():

#             predictions = (
#                 XY_grp
#                 .copy()
#                 .assign(yhat_trend_log = lambda df_: SEGMENTS_TREND_MODELS_TRAINSET[grp].predict(df_))
#                 .assign(num_sold_log_detrend = lambda df_: df_['num_sold_log'] - df_['yhat_trend_log'])
#                 )

#             remainder_feature_transformer = ColumnTransformer([
#                 ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_SEASONALITY_TO_ONEHOT),
#                 # no transforms required
#                 (
#                     'select_others', 
#                     'passthrough', 
#                     FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
#                 )
#                 ],
#                 verbose_feature_names_out=False,
#                 remainder='drop'
#                 ).set_output(transform='pandas')
            
#             X_remainder_train = remainder_feature_transformer.fit_transform(predictions.loc[is_training])        
#             dtrain = xgb.DMatrix(X_remainder_train, label=predictions.loc[is_training, 'num_sold_log_detrend'])
#             model_remainder = xgb.train(param, dtrain, ROUNDS_COUNT)

#             dtest = xgb.DMatrix(remainder_feature_transformer.transform(predictions))
#             yhat_remainder_log = model_remainder.predict(dtest)
#             predictions = (
#                 predictions
#                 .assign(yhat_remainder_log = yhat_remainder_log)
#                 .assign(yhat = lambda df_: np.exp(df_['yhat_trend_log'] + df_['yhat_remainder_log']))
#                 )

#             segments_predictions.append(predictions)

#         predictions = pd.concat(segments_predictions, axis=0)

#         scores = {
#             'validation': {
#                 'nobs': predictions.loc[is_validation].shape[0],
#                 'score': mean_absolute_percentage_error( 
#                     predictions.loc[is_validation, 'num_sold'],
#                     predictions.loc[is_validation, 'yhat']
#                     )
#                 },
#             'train': {
#                 'nobs': predictions.loc[is_training].shape[0],
#                 'score': mean_absolute_percentage_error( 
#                     predictions.loc[is_training, 'num_sold'],
#                     predictions.loc[is_training, 'yhat']
#                     )
#                 }
#             }
        
#         kfolds_evaluation.append(scores)

#     score_overall = kfolds_evaluation[0]['validation']['score']

#     return score_overall


# study = optuna.create_study()
# # if optuna returns nulls in y_pred, don't fail the entire study
# study.optimize(
#     objective,
#     n_trials=10,
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=12 * 60 * 60,
# )

In [60]:
# {'max_depth': 1, 'subsample': 0.9499378306625286, 'colsample_bytree': 0.8295502726981714}

### Overall Model: Parametric Stepwise Features Selection

In [ ]:
# SEGMENTS_MODELS_TRAINSET = {}

# kfolds_evaluation = []
# for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

#     segments_predictions = []

#     i = 0
#     segments_total = len(segments_XY_transform_features_universe)
#     for grp in segments_XY_transform_features_universe.keys():

#         print(f"{grp} modeling begun.")

#         XY_grp = segments_XY_transform_features_universe[grp]['XY_grp']
#         # when a particular series has null outcomes, time categoricals may be missing.
#         features_universe = list(segments_XY_transform_features_universe[grp]['X_train'].columns)
        
#         model_champion = Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS)
#         model_champion.fit(
#             XY_grp.loc[is_training, ['gdp_per_capita_log']],
#             XY_grp.loc[is_training, 'num_sold_log']
#             )
#         features_champion = model_champion.feature_names_in_

#         predictions_champion = (
#             XY_grp
#             .copy()
#             .assign(yhat = lambda df_: np.exp(model_champion.predict(df_[features_champion])))
#             )
#         score_champion = mean_absolute_percentage_error(
#             predictions_champion.loc[is_validation, 'num_sold'],
#             predictions_champion.loc[is_validation, 'yhat']
#             )
        
#         is_champion_beatable = True

#         while is_champion_beatable:

#             models_challengers = []
#             predictions_challengers = []
#             scores_challengers = []
#             features_champion = model_champion.feature_names_in_
#             features_marginal = list( set(features_universe).difference(set(features_champion)) )
#             for feature_mrgn in features_marginal:

#                 features_challenger = list(features_champion) + [feature_mrgn]
#                 model_challenger = Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS)
#                 model_challenger.fit(
#                     XY_grp.loc[is_training, features_challenger],
#                     XY_grp.loc[is_training, 'num_sold_log']
#                     )

#                 models_challengers.append(model_challenger)

#                 predictions_challenger = (
#                     XY_grp
#                     .copy()
#                     .assign(yhat = lambda df_: np.exp(model_challenger.predict(df_[features_challenger])))
#                     )
#                 predictions_challengers.append(predictions_challenger)

#                 score_challenger = mean_absolute_percentage_error(
#                     predictions_challenger.loc[is_validation, 'num_sold'],
#                     predictions_challenger.loc[is_validation, 'yhat']
#                     )
#                 scores_challengers.append(score_challenger)

#             score_best_challenger = min(scores_challengers)
#             is_best_challenger_better_vs_champion = score_best_challenger < score_champion
#             if is_best_challenger_better_vs_champion:
#                 index_best = scores_challengers.index(score_best_challenger)
#                 model_champion = models_challengers[index_best]
#                 score_champion = scores_challengers[index_best]            
#                 predictions_champion = predictions_challengers[index_best]
#             else:
#                 is_champion_beatable = False
            

#         SEGMENTS_MODELS_TRAINSET[grp] = model_champion

#         segments_predictions.append(predictions_champion)
        
#         i += 1
#         print(f"{i}/{segments_total} models fit.")

#     predictions = pd.concat(segments_predictions, axis=0)

#     scores = {
#         'validation': {
#             'nobs': predictions.loc[is_validation].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_validation, 'num_sold'],
#                 predictions.loc[is_validation, 'yhat']
#                 )
#             },
#         'train': {
#             'nobs': predictions.loc[is_training].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_training, 'num_sold'],
#                 predictions.loc[is_training, 'yhat']
#                 )
#             }
#         }
    
#     kfolds_evaluation.append(scores)

# kfolds_evaluation

In [ ]:
segments_scores = []
SEGMENTS_FEATURES_SELECTED = {}

segments_XY_sample = {
    # 'Canada|Discount Stickers|Kaggle': segments_XY['Canada|Discount Stickers|Kaggle'],
    # 'Canada|Discount Stickers|Kaggle Tiers': segments_XY['Canada|Discount Stickers|Kaggle Tiers']
    'Kenya|Stickers for Less|Holographic Goose': segments_XY['Kenya|Stickers for Less|Holographic Goose']
    }

kfold_alternative_latest = {'validate_2016': KFOLDS_ALTERNATIVES['validate_2016']}

i = 0
segments_total = len(segments_XY)
for grp, XY_grp in segments_XY.items():

    print(f"{grp} modeling begun.")

    # for a summary score robust to any one set's idiosyncrasies -- 
    # score multiple test sets
    scores_champion = []
    for kfold, (is_training, is_validation) in kfold_alternative_latest.items():

        features_champion = ['gdp_per_capita_log']
        feature_transform_pipeline = ColumnTransformer(
            [('standardizer', StandardScaler(), ['gdp_per_capita_log'])],
            verbose_feature_names_out=False,
            remainder='drop'
            ).set_output(transform='pandas')
        
        pipeline_e2e = Pipeline([
            ('transform_features', feature_transform_pipeline), 
            ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
            ])
        pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])
        model_champion = pipeline_e2e.named_steps.model

        predictions_champion = (
            XY_grp
            .copy()
            .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))
            )
        score_champion = mean_absolute_percentage_error( 
            predictions_champion.loc[is_validation, 'num_sold'],
            predictions_champion.loc[is_validation, 'yhat']
            )
        
        scores_champion.append(score_champion)
        
    score_champion = np.mean(scores_champion)

    # ultimate goal: evaluate challenger models, feature-by-feature (marginally).
    # expect features universe varies slightly by time series segment.
    # example, select weeks may have null outcomes, thus excluding those weeks as categorical levels
    feature_transform_pipeline_universe = ColumnTransformer(
        [
            ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
            ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
            ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
        ],
        verbose_feature_names_out=False,
        remainder='drop'
        ).set_output(transform='pandas')
    # most restrictive training set
    is_training_min = XY_grp['date'] < pd.to_datetime("2016-01-01")
    XY_grp_features_universe = feature_transform_pipeline_universe.fit_transform(XY_grp.loc[is_training_min])
    features_universe = list(XY_grp_features_universe.columns)

    is_champion_beatable = True

    while is_champion_beatable:

        # one "challenger" model defined by one features set.
        # compare several challengers (feature-sets); then, proceed with the best.
        scores_challengers = []
        features_marginal = list( set(features_universe).difference(set(features_champion)) )
        for feature_mrgn in features_marginal:
                
            features_challenger = list(features_champion) + [feature_mrgn]
            # for challenger's robust evaluation, summarize multiple test set evaluations -- 
            # wash out any one set's idiosyncrasies
            scores_kfolds_challenger = []
            for kfold, (is_training, is_validation) in kfold_alternative_latest.items():

                # operationally, easier to (1) transform features universe, then (2) select subset
                feature_transform_pipeline_universe = ColumnTransformer(
                    [
                        ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
                        ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
                        ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
                    ],
                    verbose_feature_names_out=False,
                    remainder='drop'
                    ).set_output(transform='pandas')
                
                features_selector = ColumnTransformer(
                    [('select_relevant', 'passthrough', features_challenger)],
                    verbose_feature_names_out=False,
                    remainder='drop'
                    ).set_output(transform='pandas')
                
                pipeline_e2e = Pipeline([
                    ('transform_features', feature_transform_pipeline_universe), 
                    ('select_relevant_features', features_selector),
                    ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
                    ])
                pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])

                predictions = XY_grp.copy().assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))
                score_kfold = mean_absolute_percentage_error( 
                    predictions.loc[is_validation, 'num_sold'],
                    predictions.loc[is_validation, 'yhat']
                    )
                scores_kfolds_challenger.append(score_kfold)

            score_challenger = np.mean(scores_kfolds_challenger)
            scores_challengers.append(score_challenger)

        # the new champion is the best challenger
        score_best_challenger = min(scores_challengers)
        is_best_challenger_better_vs_champion = score_best_challenger < score_champion
        if is_best_challenger_better_vs_champion:
            index_best = scores_challengers.index(score_best_challenger)
            features_champion += [features_marginal[index_best]]
            score_champion = scores_challengers[index_best]
        else:
            is_champion_beatable = False

    SEGMENTS_FEATURES_SELECTED[grp] = features_champion
    segments_scores.append(score_champion)

    i += 1
    print(f"Champion features set has size {len(features_champion)}")
    print(f"{i}/{segments_total} models fit.")

np.mean(segments_scores)

In [272]:
with open("./models/segments_features_select.txt", 'w') as file:
    file.write(str(SEGMENTS_FEATURES_SELECTED))

In [252]:
# import ast

# with open("./models/segments_features_select.txt", 'r') as f:
#     SEGMENTS_FEATURES_SELECTED = ast.literal_eval(f.read())

In [273]:
has_no_gdp = ['gdp_per_capita_log' not in x for x in SEGMENTS_FEATURES_SELECTED.values()]
assert sum(has_no_gdp) == 0

In [ ]:
kfolds_evaluation = []
for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

    segments_predictions = []

    for grp, XY_grp in segments_XY.items():

        feature_transform_pipeline_universe = ColumnTransformer(
            [
                ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
                ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
                ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
            ],
            verbose_feature_names_out=False,
            remainder='drop'
            ).set_output(transform='pandas')
        
        features_selected = SEGMENTS_FEATURES_SELECTED[grp]
        features_selector = ColumnTransformer(
            [('select_relevant', 'passthrough', features_selected)],
            verbose_feature_names_out=False,
            remainder='drop'
            ).set_output(transform='pandas')
        
        pipeline_e2e = Pipeline([
            ('transform_features', feature_transform_pipeline_universe), 
            ('select_relevant_features', features_selector),
            ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
            ])
        pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])

        predictions = XY_grp.copy().assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))

        segments_predictions.append(predictions)

    predictions = pd.concat(segments_predictions, axis=0)

    scores = {
        'validation': {
            'nobs': predictions.loc[is_validation].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                )
            },
        'train': {
            'nobs': predictions.loc[is_training].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        }
    
    kfolds_evaluation.append(scores)

kfolds_evaluation

In [263]:
predictions = (
    predictions
    .assign(ape = lambda df_: abs(100 * (df_['num_sold'] / df_['yhat'] - 1)))
    .assign(pe = lambda df_: (100 * (df_['num_sold'] / df_['yhat'] - 1)))
    .assign(residual = lambda df_: df_['num_sold'] - df_['yhat'])
    .assign(residual_log = lambda df_: df_['num_sold_log'] - np.log(df_['yhat']))
    .assign(residual_abs = lambda df_: np.abs(df_['residual']))
)

In [ ]:
(
    predictions
    .groupby('country')
    ['ape']
    .agg(['mean', 'size'])
)

In [ ]:
(
    predictions
    .groupby('year')
    ['ape']
    .agg(['mean', 'size'])
)

In [ ]:
(
    predictions
    .groupby(['country', 'year'])
    ['ape']
    .agg(['mean', 'size'])
    .unstack(-1)
)

In [ ]:
predictions_sample = predictions.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line()

In [ ]:
def objective(trial):

    ROUNDS_COUNT = 1_000

    param = {
        "objective": "reg:squarederror",
        "booster": "gbtree",
        "max_depth": trial.suggest_int("max_depth", 1, 9, step=2),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        }

    kfolds_evaluation = []
    for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

        segments_models = {}
        segments_predictions = []

        for grp, XY_grp in segments_XY.items():

            trend_level_feature_transformer = ColumnTransformer(
                [('transformer_std', StandardScaler(), ['gdp_per_capita_log'])],
                verbose_feature_names_out=False,
                remainder='drop'
                ).set_output(transform='pandas')
            trend_pipeline = Pipeline([
                ('feature_transform_pipeline', trend_level_feature_transformer),
                ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
                ])
            trend_pipeline.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])

            predictions = (
                XY_grp
                .copy()
                .assign(yhat_trend_log = lambda df_: trend_pipeline.predict(df_))
                .assign(num_sold_log_detrend = lambda df_: df_['num_sold_log'] - df_['yhat_trend_log'])
                )
            

            feature_universe_transformer = ColumnTransformer(
                [
                    ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
                    ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
                    ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
                ],
                verbose_feature_names_out=False,
                remainder='drop'
                ).set_output(transform='pandas')
            features_selected = SEGMENTS_FEATURES_SELECTED[grp].copy()
            # TODO: delete could fail due to data copies across parallel threads?
            if 'gdp_per_capita_log' in features_selected:
                features_selected.remove('gdp_per_capita_log')
            features_selector = ColumnTransformer(
                [('select_relevant', 'passthrough', features_selected)],
                verbose_feature_names_out=False,
                remainder='drop'
                ).set_output(transform='pandas')
            remainder_feature_transformer = Pipeline([
                ('transform_features', feature_universe_transformer), 
                ('select_relevant_features', features_selector)
                ])
            
            X_remainder_train = remainder_feature_transformer.fit_transform(predictions.loc[is_training])        
            dtrain = xgb.DMatrix(X_remainder_train, label=predictions.loc[is_training, 'num_sold_log_detrend'])
            model_remainder = xgb.train(param, dtrain, ROUNDS_COUNT)

            dtest = xgb.DMatrix(remainder_feature_transformer.transform(predictions))
            yhat_remainder_log = model_remainder.predict(dtest)
            predictions = (
                predictions
                .assign(yhat_remainder_log = yhat_remainder_log)
                .assign(yhat = lambda df_: np.exp(df_['yhat_trend_log'] + df_['yhat_remainder_log']))
                )

            segments_predictions.append(predictions)

        predictions = pd.concat(segments_predictions, axis=0)

        scores = {
            'validation': {
                'nobs': predictions.loc[is_validation].shape[0],
                'score': mean_absolute_percentage_error( 
                    predictions.loc[is_validation, 'num_sold'],
                    predictions.loc[is_validation, 'yhat']
                    )
                },
            'train': {
                'nobs': predictions.loc[is_training].shape[0],
                'score': mean_absolute_percentage_error( 
                    predictions.loc[is_training, 'num_sold'],
                    predictions.loc[is_training, 'yhat']
                    )
                }
            }
        
        kfolds_evaluation.append(scores)

    score_overall = kfolds_evaluation[0]['validation']['score']

    return score_overall


study = optuna.create_study()
# if optuna returns nulls in y_pred, don't fail the entire study
study.optimize(
    objective,
    n_trials=10,
    catch=(ValueError,),
    n_jobs=-1,
    timeout=12 * 60 * 60,
)

### Seasonal & Remainder Component: Nonparametric, Fix Params

In [ ]:
# # from preceding optuna run
# REMAINDER_MODEL_XGBOOST_ROUNDS_COUNT = 1_000
# # TODO: could deeper optimization help?
# REMAINDER_XGBOOST_MODEL_BEST_PARAMS = {
#     "objective": "reg:squarederror",
#     "booster": "gbtree",
#     'max_depth': 1,
#     'subsample': 0.9,
#     'colsample_bytree': 0.5
#     }

# kfolds_evaluation = []
# for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

#     segments_models = {}
#     segments_predictions = []

#     for grp, XY_grp in segments_XY.items():

#         trend_level_model_features_selected = SEGMENTS_TREND_MODELS_TRAINSET[grp].named_steps.model
#         trend_level_feature_transformer = ColumnTransformer(
#             [('transformer_std', StandardScaler(), trend_level_model_features_selected.feature_names_in_)],
#             verbose_feature_names_out=False,
#             remainder='drop'
#             ).set_output(transform='pandas')

#         remainder_feature_transformer = ColumnTransformer([
#             ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_SEASONALITY_TO_ONEHOT),
#             # no transforms required
#             (
#                 'select_others', 
#                 'passthrough', 
#                 FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
#             )
#             ],
#             verbose_feature_names_out=False,
#             remainder='drop'
#             ).set_output(transform='pandas')

#         pipeline_e2e = TrendRemainderModelPipeline(
#             trend_level_feature_transformer,
#             RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS['alpha'],
#             remainder_feature_transformer, 
#             'xgboost',
#             REMAINDER_MODEL_XGBOOST_ROUNDS_COUNT, 
#             REMAINDER_XGBOOST_MODEL_BEST_PARAMS
#             )

#         pipeline_e2e.fit(XY_grp.loc[is_training], XY_grp.loc[is_training, 'num_sold_log'])
        
#         segments_models[grp] = pipeline_e2e

#         predictions = (
#             XY_grp
#             .copy()
#             .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))
#             )
#         segments_predictions.append(predictions)

#     predictions = pd.concat(segments_predictions, axis=0)

#     scores = {
#         'validation': {
#             'nobs': predictions.loc[is_validation].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_validation, 'num_sold'],
#                 predictions.loc[is_validation, 'yhat']
#                 )
#             },
#         'train': {
#             'nobs': predictions.loc[is_training].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_training, 'num_sold'],
#                 predictions.loc[is_training, 'yhat']
#                 )
#             }
#         }
    
#     kfolds_evaluation.append(scores)

# kfolds_evaluation

In [154]:
# predictions = (
#     predictions
#     .assign(ape = lambda df_: abs(100 * (df_['num_sold'] / df_['yhat'] - 1)))
#     .assign(pe = lambda df_: (100 * (df_['num_sold'] / df_['yhat'] - 1)))
#     .assign(residual = lambda df_: df_['num_sold'] - df_['yhat'])
#     .assign(residual_log = lambda df_: df_['num_sold_log'] - np.log(df_['yhat']))
#     .assign(residual_abs = lambda df_: np.abs(df_['residual']))
# )

In [ ]:
# (
#     predictions
#     .groupby('country')
#     ['ape']
#     .agg(['mean', 'size'])
# )

In [ ]:
# (
#     predictions
#     .groupby('year')
#     ['ape']
#     .agg(['mean', 'size'])
# )

In [ ]:
# (
#     predictions
#     .groupby(['country', 'year'])
#     ['ape']
#     .agg(['mean', 'size'])
#     .unstack(-1)
# )

In [ ]:
# model_sample = segments_models['Kenya|Stickers for Less|Holographic Goose']
# importances = pd.DataFrame.from_dict(model_sample.remainder_model.get_score(importance_type='gain'), orient='index')
# importances.sort_values(0, ascending=False)

In [ ]:
# predictions_sample = predictions.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

# predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line()

In [ ]:
# predictions_sample = (
#     predictions_sample
#     .assign(residual = lambda df_: df_['num_sold'] - df_['yhat'])
#     .sort_values('date')
#     .assign(
#         residual_lag1 = lambda df_: df_['residual'].shift(1),
#         residual_lag2 = lambda df_: df_['residual'].shift(2),
#         residual_lag3 = lambda df_: df_['residual'].shift(3),
#         residual_lag5 = lambda df_: df_['residual'].shift(5),
#         residual_lag7 = lambda df_: df_['residual'].shift(7),
#         residual_lag14 = lambda df_: df_['residual'].shift(14),
#         )
#     )

# (
#     p9.ggplot((
#         predictions_sample
#         .loc[predictions_sample['date'] <= pd.to_datetime("2014-01-01")]
#         )) + 
#     p9.theme_bw() + 
#     p9.geom_point(p9.aes('residual_lag1', 'residual')) + 
#     p9.geom_smooth(p9.aes('residual_lag1', 'residual'), method='lm', se=False, color='lightblue')
# )

In [162]:
# (
#     predictions
#     .assign(mape = lambda df_: np.abs(100 * (df_['num_sold']/df_['yhat'] - 1)) )
#     .sort_values('mape', ascending=False)
#     .head(100)
#     .to_csv("./data/processed/predictions_errors_local_model.csv", index=False)
# )

In [163]:
# (
#     predictions
#     .assign(mape = lambda df_: np.abs(100 * (df_['num_sold']/df_['yhat'] - 1)) )
#     .sort_values('mape', ascending=False)
#     .head(50)
#     .to_csv("./data/processed/predictions_local_model.csv")
# )

In [ ]:
# predictions_sample = predictions.query("series_id == 'Kenya|Stickers for Less|Holographic Goose'")

# predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line()

### Ultimate Models Fit

In [177]:
# SEGMENTS_MODELS = {}

# for grp, XY_grp in segments_XY.items():

#     feature_transform_pipeline_universe = ColumnTransformer(
#         [
#             ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_LOCAL_MODEL_TO_ONEHOT),
#             ('standardizer', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS),
#             ('select_others', 'passthrough', FEATURES_UNIVERSE_ALREADY_ONEHOT)
#         ],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')
    
#     features_selected = SEGMENTS_FEATURES_SELECTED[grp]
#     features_selector = ColumnTransformer(
#         [('select_relevant', 'passthrough', features_selected)],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')
    
#     pipeline_e2e = Pipeline([
#         ('transform_features', feature_transform_pipeline_universe), 
#         ('select_relevant_features', features_selector),
#         ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS))
#         ])
#     pipeline_e2e.fit(XY_grp, XY_grp['num_sold_log'])
    
#     SEGMENTS_MODELS[grp] = pipeline_e2e

In [178]:
# SEGMENTS_MODELS = {}

# for grp, XY_grp in segments_XY.items():

#     trend_level_model_features_selected = SEGMENTS_TREND_MODELS_TRAINSET[grp].named_steps.model
#     trend_level_feature_transformer = ColumnTransformer(
#         [('transformer_std', StandardScaler(), trend_level_model_features_selected.feature_names_in_)],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')

#     remainder_feature_transformer = ColumnTransformer([
#         ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_SEASONALITY_TO_ONEHOT),
#         (
#             'select_others', 
#             'passthrough', 
#             FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
#         )
#         ],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')

#     pipeline_e2e = TrendRemainderModelPipeline(
#         trend_level_feature_transformer,
#         RIDGE_ALPHA_TREND_CYCLE_LOCAL_MODELS['alpha'],
#         remainder_feature_transformer, 
#         'xgboost',
#         REMAINDER_MODEL_XGBOOST_ROUNDS_COUNT, 
#         REMAINDER_XGBOOST_MODEL_BEST_PARAMS
#         )
#     pipeline_e2e.fit(XY_grp, XY_grp['num_sold_log'])
    
#     SEGMENTS_MODELS[grp] = pipeline_e2e

## Global Model

### Nonparametric

In [ ]:
kfolds_evaluation = []
for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

    feature_transform_pipeline_global_model = ColumnTransformer([
        ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
        ('transformer_std', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS)
        ],
        verbose_feature_names_out=False,
        # caller expected to provide some features that require no transforms.
        # don't drop those just because they received no special operations.
        # but do be sure to drop non-features in advance!
        remainder='passthrough'
        )
    feature_transform_pipeline_global_model.set_output(transform='pandas')
    
    pipeline_e2e = Pipeline([
        ('transform_features', feature_transform_pipeline_global_model), 
        ('model', RandomForestRegressor(n_estimators=100, n_jobs=-1))
        ])

    pipeline_e2e.fit(
        XY.loc[is_training, FEATURES_GLOBAL_MODEL],
        XY.loc[is_training, 'num_sold_log']
        )
    
    predictions = (
        XY
        .copy()
        .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_[FEATURES_GLOBAL_MODEL])))
        )
    
    scores = {
        'validation': {
            'nobs': predictions.loc[is_validation].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                )
            },
        'train': {
            'nobs': predictions.loc[is_training].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        }
    
    kfolds_evaluation.append(scores)

kfolds_evaluation

In [ ]:
predictions_sample = predictions.query("series_id == 'Norway|Stickers for Less|Kaggle'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line(alpha=0.5);

### Parametric

In [ ]:
kfolds_evaluation = []
for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

    feature_transform_pipeline_global_model = ColumnTransformer([
        ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
        ('transformer_std', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS)
        ],
        verbose_feature_names_out=False,
        remainder='passthrough'
        )
    feature_transform_pipeline_global_model.set_output(transform='pandas')

    pipeline_e2e = Pipeline([
        ('transform_features', feature_transform_pipeline_global_model), 
        ('model', Ridge(1e-2))
        ])

    pipeline_e2e.fit(
        XY.loc[is_training, FEATURES_GLOBAL_MODEL],
        XY.loc[is_training, 'num_sold_log']
        )
    
    predictions = (
        XY
        .copy()
        .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_[FEATURES_GLOBAL_MODEL])))
        )
    
    scores = {
        'validation': {
            'nobs': predictions.loc[is_validation].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                )
            },
        'train': {
            'nobs': predictions.loc[is_training].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        }
    
    kfolds_evaluation.append(scores)

kfolds_evaluation

In [ ]:
predictions_sample = predictions.query("series_id == 'Norway|Stickers for Less|Kaggle'")
# predictions_sample = predictions.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line(alpha=0.5);

### Trend-Cycle Component: Parametric, Explore Params

In [ ]:
FEATURES_TREND_CYCLE_TO_ONEHOT = [
    'country', 
    'store',
    'product',
    'country_store',
    'country_product',
    'store_product',
    ]

def objective(trial):

    parameters = {'alpha': trial.suggest_float("alpha", 1e-1, 1_000, log=True)}

    kfolds_evaluation = []
    for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

        feature_transform_pipeline_global_model = ColumnTransformer([
            ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_TREND_CYCLE_TO_ONEHOT),
            ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
            ],
            verbose_feature_names_out=False,
            remainder='drop'
            )
        feature_transform_pipeline_global_model.set_output(transform='pandas')

        pipeline_e2e = Pipeline([
            ('transform_features', feature_transform_pipeline_global_model), 
            ('model', Ridge(**parameters))
            ])

        pipeline_e2e.fit(XY.loc[is_training], XY.loc[is_training, 'num_sold_log'])
        
        predictions = (
            XY
            .copy()
            .assign(yhat = lambda df_: np.exp(pipeline_e2e.predict(df_)))
            )
        
        scores = {
            'validation': {
                'nobs': predictions.loc[is_validation].shape[0],
                'score': mean_absolute_percentage_error( 
                    predictions.loc[is_validation, 'num_sold'],
                    predictions.loc[is_validation, 'yhat']
                    )
                },
            'train': {
                'nobs': predictions.loc[is_training].shape[0],
                'score': mean_absolute_percentage_error( 
                    predictions.loc[is_training, 'num_sold'],
                    predictions.loc[is_training, 'yhat']
                    )
                }
            }
        
        kfolds_evaluation.append(scores)

    score_overall = np.mean([ 
        kfolds_evaluation[0]['validation']['score']
        ])
    
    return score_overall

study = optuna.create_study()

study.optimize(
    objective,
    n_trials=50,
    catch=(ValueError,),
    n_jobs=-1,
    timeout=12 * 60 * 60,
)

RIDGE_ALPHA_TREND_CYCLE_GLOBAL_MODEL = study.best_params

In [ ]:
study.best_trial, study.best_params, study.best_value

### Seasonal & Remainder Component: Nonparametric, Explore Params

Curiously, haven't improved global model by implementing xgboost remainder model.

In [186]:
# def objective(trial):

#     ROUNDS_COUNT = 1_000

#     param = {
#         "objective": "reg:squarederror",
#         "booster": "gbtree",
#         "max_depth": trial.suggest_int("max_depth", 1, 9, step=2),
#         "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         }

#     kfolds_evaluation = []
#     for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

#         feature_transform_pipeline_trend_model = ColumnTransformer([
#             ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_TREND_CYCLE_TO_ONEHOT),
#             ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
#             ],
#             verbose_feature_names_out=False,
#             remainder='drop'
#             ).set_output(transform='pandas')
        
#         pipeline_trend_e2e = Pipeline([
#             ('transform_features', feature_transform_pipeline_trend_model), 
#             ('model', Ridge(**RIDGE_ALPHA_TREND_CYCLE_GLOBAL_MODEL))
#             ])
#         pipeline_trend_e2e.fit(XY.loc[is_training], XY.loc[is_training, 'num_sold_log'])

#         predictions = (
#             XY
#             .copy()
#             .assign(yhat_trend_log = lambda df_: pipeline_trend_e2e.predict(df_))
#             .assign(num_sold_log_detrend = lambda df_: df_['num_sold_log'] - df_['yhat_trend_log'])
#             )


#         remainder_feature_transformer = ColumnTransformer([
#             ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_SEASONALITY_TO_ONEHOT),
#             # no transforms required
#             (
#                 'select_others', 
#                 'passthrough', 
#                 FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT + ['yhat_trend_log']
#             )
#             ],
#             verbose_feature_names_out=False,
#             remainder='drop'
#             ).set_output(transform='pandas')
        
#         X_remainder_train = remainder_feature_transformer.fit_transform(predictions.loc[is_training])        
#         dtrain = xgb.DMatrix(X_remainder_train, label=predictions.loc[is_training, 'num_sold_log_detrend'])
#         model_remainder = xgb.train(param, dtrain, ROUNDS_COUNT)

#         dtest = xgb.DMatrix(remainder_feature_transformer.transform(predictions))
#         yhat_remainder_log = model_remainder.predict(dtest)
#         predictions = (
#             predictions
#             .assign(yhat_remainder_log = yhat_remainder_log)
#             .assign(yhat = lambda df_: np.exp(df_['yhat_trend_log'] + df_['yhat_remainder_log']))
#             )


#         scores = {
#             'validation': {
#                 'nobs': predictions.loc[is_validation].shape[0],
#                 'score': mean_absolute_percentage_error( 
#                     predictions.loc[is_validation, 'num_sold'],
#                     predictions.loc[is_validation, 'yhat']
#                     )
#                 },
#             'train': {
#                 'nobs': predictions.loc[is_training].shape[0],
#                 'score': mean_absolute_percentage_error( 
#                     predictions.loc[is_training, 'num_sold'],
#                     predictions.loc[is_training, 'yhat']
#                     )
#                 }
#             }
        
#         kfolds_evaluation.append(scores)

#     score_overall = kfolds_evaluation[0]['validation']['score']

#     return score_overall


# study = optuna.create_study()
# # if optuna returns nulls in y_pred, don't fail the entire study
# study.optimize(
#     objective,
#     n_trials=10,
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=12 * 60 * 60,
# )

In [187]:
# REMAINDER_XGBOOST_GLOBAL_MODEL_BEST_PARAMS = {
#     "objective": "reg:squarederror",
#     "booster": "gbtree",
#     'max_depth': 1,
#     'subsample': 0.64,
#     'colsample_bytree': 0.6
#     }

# kfolds_evaluation = []
# for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

#     trend_level_feature_transformer = ColumnTransformer([
#         ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_TREND_CYCLE_TO_ONEHOT),
#         ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
#         ],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')

#     remainder_feature_transformer = ColumnTransformer([
#         ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
#         # no transforms required
#         (
#             'select_others', 
#             'passthrough', 
#             FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
#         )
#         ],
#         verbose_feature_names_out=False,
#         remainder='drop'
#         ).set_output(transform='pandas')

#     model_global_pipeline = TrendRemainderModelPipeline(
#         trend_level_feature_transformer,
#         RIDGE_ALPHA_TREND_CYCLE_GLOBAL_MODEL['alpha'],
#         remainder_feature_transformer,
#         'xgboost',
#         1_000,
#         REMAINDER_XGBOOST_GLOBAL_MODEL_BEST_PARAMS
#         )
#     model_global_pipeline.fit(XY.loc[is_training], XY.loc[is_training, 'num_sold_log'])
    
#     predictions = XY.copy()
#     predictions = (
#         predictions
#         .assign(yhat_log = lambda df_: model_global_pipeline.predict(df_))
#         .assign(yhat = lambda df_: np.exp(df_['yhat_log']) )
#         )
    
#     scores = {
#         'validation': {
#             'nobs': predictions.loc[is_validation].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_validation, 'num_sold'],
#                 predictions.loc[is_validation, 'yhat']
#                 )
#             },
#         'train': {
#             'nobs': predictions.loc[is_training].shape[0],
#             'score': mean_absolute_percentage_error( 
#                 predictions.loc[is_training, 'num_sold'],
#                 predictions.loc[is_training, 'yhat']
#                 )
#             }
#         }
    
#     kfolds_evaluation.append(scores)

# kfolds_evaluation

### Seasonal & Remainder Component: Nonparametric, Fix Params

In [ ]:
kfolds_evaluation = []
for is_training, is_validation in [KFOLDS_ALTERNATIVES['validate_2016']]:

    trend_level_feature_transformer = ColumnTransformer([
        ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_TREND_CYCLE_TO_ONEHOT),
        ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
        ],
        verbose_feature_names_out=False,
        remainder='drop'
        ).set_output(transform='pandas')

    remainder_feature_transformer = ColumnTransformer([
        ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
        # no transforms required
        (
            'select_others', 
            'passthrough', 
            FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
        )
        ],
        verbose_feature_names_out=False,
        remainder='drop'
        ).set_output(transform='pandas')

    model_global_pipeline = TrendRemainderModelPipeline(
        trend_level_feature_transformer,
        RIDGE_ALPHA_TREND_CYCLE_GLOBAL_MODEL['alpha'],
        remainder_feature_transformer,
        'random_forest'
        )
    model_global_pipeline.fit(XY.loc[is_training], XY.loc[is_training, 'num_sold_log'])
    
    predictions = XY.copy()
    predictions = (
        predictions
        .assign(yhat_log = lambda df_: model_global_pipeline.predict(df_))
        .assign(yhat = lambda df_: np.exp(df_['yhat_log']) )
        )
    
    scores = {
        'validation': {
            'nobs': predictions.loc[is_validation].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_validation, 'num_sold'],
                predictions.loc[is_validation, 'yhat']
                )
            },
        'train': {
            'nobs': predictions.loc[is_training].shape[0],
            'score': mean_absolute_percentage_error( 
                predictions.loc[is_training, 'num_sold'],
                predictions.loc[is_training, 'yhat']
                )
            }
        }
    
    kfolds_evaluation.append(scores)

kfolds_evaluation

In [ ]:
(
    predictions
    .assign(ape = lambda df_: abs(100 * (df_['num_sold'] / df_['yhat'] - 1)))
    .groupby('country')
    ['ape']
    .agg(['mean', 'size'])
)

In [ ]:
predictions_sample = predictions.query("series_id == 'Norway|Stickers for Less|Kaggle'")
# predictions_sample = predictions.query("series_id == 'Norway|Premium Sticker Mart|Kaggle'")

predictions_sample.set_index('date')[['num_sold', 'yhat']].plot.line(alpha=0.5);

In [191]:
# WIP

# kfolds = [

#     # validation set 2014-16 matches ultimate test set's length, 2017-19
#     ( 
#         ( (XY['date'] >= pd.to_datetime('2010-01-01')) & (XY['date'] < pd.to_datetime("2014-01-01")) ),
#         XY['date'] >= pd.to_datetime("2014-01-01")
#     ),

#     # ( 
#     #     ( (XY['date'] >= pd.to_datetime('2010-01-01')) & (XY['date'] < pd.to_datetime("2016-01-01")) ),
#     #     XY['date'] >= pd.to_datetime("2016-01-01")
#     # )

#     ]

# def objective(trial):

#     ROUNDS_COUNT = 1_000

#     param = {
#         "objective": "reg:squarederror",
#         "booster": trial.suggest_categorical(
#             "booster", ["gbtree", "gblinear", "dart"]
#         ),
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         'device': 'cuda',
#         'tree_method': 'hist'
#     }

#     if param["booster"] in ["gbtree", "dart"]:
#         param["max_depth"] = trial.suggest_int("max_depth", 1, 9, step=2)

#         param["min_child_weight"] = trial.suggest_int("min_child_weight", 1, 10)
#         param["eta"] = trial.suggest_float("eta", 1e-5, 0.01, log=True)

#         param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
#         param["grow_policy"] = trial.suggest_categorical(
#             "grow_policy", ["depthwise", "lossguide"]
#         )

#     if param["booster"] == "dart":
#         param["sample_type"] = trial.suggest_categorical(
#             "sample_type", ["uniform", "weighted"]
#         )
#         param["normalize_type"] = trial.suggest_categorical(
#             "normalize_type", ["tree", "forest"]
#         )
#         param["rate_drop"] = trial.suggest_float(
#             "rate_drop", 1e-8, 1.0, log=True
#         )
#         param["skip_drop"] = trial.suggest_float(
#             "skip_drop", 1e-8, 1.0, log=True
#         )


#     kfolds_evaluation = []
#     for is_training, is_validation in kfolds:

#         feature_transform_pipeline_global_model = ColumnTransformer([
#             ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
#             ('transformer_std', StandardScaler(), FEATURES_UNIVERSE_NUMERIC_CONTINUOUS)
#             ],
#             verbose_feature_names_out=False,
#             # caller expected to provide some features that require no transforms.
#             # don't drop those just because they received no special operations.
#             # but do be sure to drop non-features in advance!
#             remainder='passthrough'
#             )
#         feature_transform_pipeline_global_model.set_output(transform='pandas')

#         X_train = (
#             feature_transform_pipeline_global_model
#             .fit_transform(XY.loc[is_training, FEATURES_GLOBAL_MODEL])
#             )
#         dtrain = xgb.DMatrix(
#             X_train, 
#             label=XY.loc[is_training, 'num_sold_log']
#             )
        
#         X_test = feature_transform_pipeline_global_model.transform(XY)
#         dtest = xgb.DMatrix(X_test)

#         model_global = xgb.train(param, dtrain, ROUNDS_COUNT)
        
#         yhat_log = model_global.predict(dtest)
#         predictions = (
#             XY
#             .copy()
#             .assign(yhat = np.exp(yhat_log))
#             )

#         scores = {
#             'validation': mean_absolute_percentage_error( 
#                 predictions.loc[is_validation, 'num_sold'],
#                 predictions.loc[is_validation, 'yhat']
#                 ),
#             'train': mean_absolute_percentage_error( 
#                 predictions.loc[is_training, 'num_sold'],
#                 predictions.loc[is_training, 'yhat']
#                 )
#             }
        
#         kfolds_evaluation.append(scores)

#     score_overall = np.mean([ 
#         kfolds_evaluation[0]['validation'], 
#         # kfolds_evaluation[1]['validation'] 
#         ])
    
#     return score_overall

# study = optuna.create_study()

# study.optimize(
#     objective,
#     n_trials=50,
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=12 * 60 * 60,
# )

In [ ]:
trend_level_feature_transformer = ColumnTransformer([
    ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_TREND_CYCLE_TO_ONEHOT),
    ('transformer_std', StandardScaler(), FEATURES_TREND_CYCLE_NUMERIC_CONTINUOUS)
    ],
    verbose_feature_names_out=False,
    remainder='drop'
    ).set_output(transform='pandas')

remainder_feature_transformer = ColumnTransformer([
    ('transformer_onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), FEATURES_UNIVERSE_TO_ONEHOT),
    # no transforms required
    (
        'select_others', 
        'passthrough', 
        FEATURES_SEASONALITY_NUMERIC_CONTINUOUS + FEATURES_UNIVERSE_ALREADY_ONEHOT
    )
    ],
    verbose_feature_names_out=False,
    remainder='drop'
    ).set_output(transform='pandas')

MODEL_GLOBAL = TrendRemainderModelPipeline(
    trend_level_feature_transformer,
    RIDGE_ALPHA_TREND_CYCLE_GLOBAL_MODEL['alpha'],
    remainder_feature_transformer,
    'random_forest'
    )

MODEL_GLOBAL.fit(XY, XY['num_sold_log'])

# Deployment

In [193]:
sales_test_daily = pd.read_csv("./data/external/test.csv").assign(

    date = lambda df_: pd.to_datetime(df_['date']),

    country_store = lambda df_: df_['country'].str.cat(df_['store'], sep='|'),
    country_product = lambda df_: df_['country'].str.cat(df_['product'], sep='|'),
    store_product = lambda df_: df_['store'].str.cat(df_['product'], sep='|'),
    country_store_product = lambda df_: df_['country'].str.cat([df_['store'], df_['product']], sep='|')

    ).assign(series_id = lambda df_: df_['country_store_product'])

In [ ]:
sales_test_daily.head()

In [ ]:
sales_test_daily['date'].describe()

In [196]:
sales_test_daily = transform_calendar_features(sales_test_daily)
sales_test_daily = integrate_external_features(sales_test_daily)
sales_test_daily = sales_test_daily.assign(
    num_sold = None,
    num_sold_log = None
    )

In [197]:
# tried lower-clipping to historical floor -- public LB suffered!
OUTCOME_HISTORICAL_MIN = 5

segments_X_test = {grp: df for grp, df in sales_test_daily.groupby('series_id')}

segments_predictions_test = []
novel_series_count = 0
for grp, df in segments_X_test.items():

    if grp in SEGMENTS_MODELS:
        df = df.assign(yhat = lambda df_: np.exp(SEGMENTS_MODELS[grp].predict(df_)))  
        
    else:
        df = df.assign(yhat = lambda df_: np.exp(MODEL_GLOBAL.predict(df_)))
        novel_series_count += 1

    segments_predictions_test.append(df)

predictions_test = pd.concat(segments_predictions_test, axis=0)

In [198]:
predictions_test_submit = (
    predictions_test
    [['id', 'yhat']]
    .rename(columns={'yhat': 'num_sold'})
    )

In [199]:
assert predictions_test_submit.shape[0] == 98_550
assert predictions_test_submit.notnull().all().all()
predictions_test_submit.to_csv("./data/processed/submission_features_selected_cv_stepwise.csv", index=False)

In [ ]:
break

## Differential Testing

In [93]:
predictions_compare = (
    pd.read_csv("./data/processed/submission_retry_features_select_trend_periodicity.csv")
    .rename(columns={'num_sold': 'num_sold_prev'})
    )
predictions_compare = (
    pd.merge(predictions_compare, predictions_test_submit, how='left')
    .assign(diff = lambda df_: 100 * (df_['num_sold'] / df_['num_sold_prev'] - 1))
    )
assert predictions_compare.shape[0] == 98_550

predictions_compare = pd.merge(
    predictions_compare, 
    sales_test_daily[['id', 'country', 'product', 'store', 'year', 'series_id']]
    )

In [ ]:
predictions_compare['diff'].describe()

In [ ]:
predictions_compare.groupby('country')['diff'].describe()

In [ ]:
predictions_compare.query("year == 2017").groupby('country')['diff'].describe()

In [ ]:
predictions_compare.groupby('year')['diff'].describe()